# NLP: User experience multilabel classification

## Описание проекта

Задача множественной классификации обратной связи от пользователей по множеству затронутых в тексте тематик

**Цель:** Разработать прототип модели, которая будет классифицировать комментарии пользователей о доставке на 50 различных классов

**Задачи:**

1) Загрузить данные, сделать обзор
2) Выполнить предобработку данных: проверить данные на дубликаты, подобрать методы обработки пропусков (если они есть) и тд.
3) Подобрать способ векторизации комментариев пользователей
4) Разработать прототип модели
5) Получить предсказания лучшей модели на тестовой выборке и отправить решение заказчику

## Предобработка данных

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import pymorphy2
import os
import pickle
import optuna
import hashlib
import sys
import requests
import tensorflow as tf
from navec import Navec



from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.utils import simple_preprocess
from gensim import utils
from gensim.test.utils import datapath
from gensim import downloader
from natasha import Doc, MorphVocab, NewsMorphTagger, NewsEmbedding, Segmenter
from joblib import Parallel, delayed

random_state=379

### Загрузка данных

In [2]:
try:
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')

    print('Загрузка данных прошла успешно')

except:
    print('Не удалось загрузить данные')

Загрузка данных прошла успешно


### Обзор данных

In [3]:
def explore_data(df):
    print("Первые 5 строк набора данных:")
    display(df.head())
    
    print("\nИнформация о наборе данных:")
    display(df.info())
    
    print("\nСтатистические характеристики числовых столбцов:")
    display(df.describe(include='all').T)
    
    print("\nЧисло уникальных значений в каждом столбце:")
    for column in df.columns:
        unique_values = df[column].unique()
        num_unique_values = len(unique_values)
        print(f"Столбец '{column}' содержит {num_unique_values} уникальных значений.")
        print()
    
    print("\nЧисло значений по категориям в каждом столбце:")
    for column in df.columns:
        if df[column].dtype == 'object':
            value_counts = df[column].value_counts()
            print(f"Столбец '{column}':")
            print(value_counts)
            print()

In [4]:
explore_data(df_train)

Первые 5 строк набора данных:


,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15237,5.0,"{ASSORTMENT,PRODUCTS_QUALITY,PROMOTIONS,CATALO...",Доставка просто 👍,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



Информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8708 entries, 0 to 8707
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           8708 non-null   int64  
 1   assessment      8478 non-null   float64
 2   tags            8697 non-null   object 
 3   text            8708 non-null   object 
 4   trend_id_res0   8708 non-null   int64  
 5   trend_id_res1   8708 non-null   int64  
 6   trend_id_res2   8708 non-null   int64  
 7   trend_id_res3   8708 non-null   int64  
 8   trend_id_res4   8708 non-null   int64  
 9   trend_id_res5   8708 non-null   int64  
 10  trend_id_res6   8708 non-null   int64  
 11  trend_id_res7   8708 non-null   int64  
 12  trend_id_res8   8708 non-null   int64  
 13  trend_id_res9   8708 non-null   int64  
 14  trend_id_res10  8708 non-null   int64  
 15  trend_id_res11  8708 non-null   int64  
 16  trend_id_res12  8708 non-null   int64  
 17  tren

None


Статистические характеристики числовых столбцов:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
index,8708.0,NaN,NaN,NaN,13438.49977,7847.950998,0.0,6685.25,13311.5,20302.5,27035.0
assessment,8478.0,NaN,NaN,NaN,4.06334,1.813434,0.0,3.0,5.0,5.0,6.0
tags,8697,1135,{DELIVERY},1137,NaN,NaN,NaN,NaN,NaN,NaN,NaN
text,8708,7483,👍,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trend_id_res0,8708.0,NaN,NaN,NaN,0.096119,0.294771,0.0,0.0,0.0,0.0,1.0
trend_id_res1,8708.0,NaN,NaN,NaN,0.039848,0.195614,0.0,0.0,0.0,0.0,1.0
trend_id_res2,8708.0,NaN,NaN,NaN,0.068558,0.252715,0.0,0.0,0.0,0.0,1.0
trend_id_res3,8708.0,NaN,NaN,NaN,0.041456,0.199354,0.0,0.0,0.0,0.0,1.0
trend_id_res4,8708.0,NaN,NaN,NaN,0.014469,0.119422,0.0,0.0,0.0,0.0,1.0
trend_id_res5,8708.0,NaN,NaN,NaN,0.005972,0.077049,0.0,0.0,0.0,0.0,1.0



Число уникальных значений в каждом столбце:
Столбец 'index' содержит 8708 уникальных значений.

Столбец 'assessment' содержит 8 уникальных значений.

Столбец 'tags' содержит 1136 уникальных значений.

Столбец 'text' содержит 7483 уникальных значений.

Столбец 'trend_id_res0' содержит 2 уникальных значений.

Столбец 'trend_id_res1' содержит 2 уникальных значений.

Столбец 'trend_id_res2' содержит 2 уникальных значений.

Столбец 'trend_id_res3' содержит 2 уникальных значений.

Столбец 'trend_id_res4' содержит 2 уникальных значений.

Столбец 'trend_id_res5' содержит 2 уникальных значений.

Столбец 'trend_id_res6' содержит 2 уникальных значений.

Столбец 'trend_id_res7' содержит 2 уникальных значений.

Столбец 'trend_id_res8' содержит 2 уникальных значений.

Столбец 'trend_id_res9' содержит 2 уникальных значений.

Столбец 'trend_id_res10' содержит 2 уникальных значений.

Столбец 'trend_id_res11' содержит 2 уникальных значений.

Столбец 'trend_id_res12' содержит 2 уникальных значений.

Сто

In [5]:
explore_data(df_test)

Первые 5 строк набора данных:


,index,assessment,tags,text
0,5905,5.0,{PROMOTIONS},"Крутая компания, удобное приложение"
1,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да..."
2,9285,5.0,"{ASSORTMENT,PROMOTIONS}",Супер!!!
3,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции
4,16778,5.0,"{ASSORTMENT,PRODUCTS_QUALITY,PRICE,PROMOTIONS,...",Отлично



Информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16999 entries, 0 to 16998
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       16999 non-null  int64  
 1   assessment  16533 non-null  float64
 2   tags        16967 non-null  object 
 3   text        16997 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 531.3+ KB


None


Статистические характеристики числовых столбцов:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
index,16999.0,NaN,NaN,NaN,13616.141773,7806.852237,1.0,6833.5,13726.0,20357.5,27039.0
assessment,16533.0,NaN,NaN,NaN,4.056554,1.827984,0.0,3.0,5.0,5.0,6.0
tags,16967,1673,{DELIVERY},2207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
text,16997,14980,.,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Число уникальных значений в каждом столбце:
Столбец 'index' содержит 16999 уникальных значений.

Столбец 'assessment' содержит 8 уникальных значений.

Столбец 'tags' содержит 1674 уникальных значений.

Столбец 'text' содержит 14981 уникальных значений.


Число значений по категориям в каждом столбце:
Столбец 'tags':
tags
{DELIVERY}                                                                 2207
{ASSORTMENT,PROMOTIONS}                                                     859
{ASSORTMENT,PRICE,PROMOTIONS}                                               713
{PROMOTIONS}                                                                704
{DELIVERY,SUPPORT}                                                          653
                                                                           ... 
{PROMOTIONS,PAYMENT,PRICE,SUPPORT}                                            1
{PROMOTIONS,PAYMENT,ASSORTMENT,PRICE,CATALOG_NAVIGATION}                      1
{DELIVERY,PRODUCTS_QUALITY,PRICE,PRO

**Комментарий:** Из первичного обзора данных можно сказать, что:  

- Столбцы 'tags' и 'assessment' содержат пропуски
- Нужно посмотреть, как значения столбцов 'tags' и 'text' зависят от значений столбца 'assessment'
- Можно перевод значений столбца 'assessment' из float в int, так как оценка не может быть вещественной.


### 'assessment' из float в int

In [6]:
df_train['assessment'] = df_train['assessment'].astype('Int64')
df_test['assessment'] = df_test['assessment'].astype('Int64')

print(df_train['assessment'].head())
print(df_test['assessment'].head())

0    6
1    4
2    6
3    6
4    5
Name: assessment, dtype: Int64
0    5
1    3
2    5
3    2
4    5
Name: assessment, dtype: Int64


### Проверка на дубликаты 

In [7]:
def dub(df):
    duplicates = df_train.duplicated()
    print(f"Найдено дубликатов: {duplicates.sum()}")

In [8]:
dub(df_train)

Найдено дубликатов: 0


In [9]:
dub(df_test)

Найдено дубликатов: 0


**Комментарий:** Отлично, явных дубликатов не найдено

### Удаление пропусков в столбцах 'tags' и 'text'

In [10]:
def drop_rows_with_missing_values(df, columns):
    initial_row_count = df.shape[0]
    missing_info = {}
    for col in columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_info[col] = {'missing_count': missing_count, 
                                 'percent': (missing_count / initial_row_count) * 100}
            df = df[df[col].notnull()].reset_index(drop=True)
    final_row_count = df.shape[0]
    removed_info = {'removed_rows': initial_row_count - final_row_count,
                    'removed_percent': ((initial_row_count - final_row_count) / initial_row_count) * 100}
    missing_info['overall'] = removed_info
    return df, missing_info

In [11]:
df_train, info = drop_rows_with_missing_values(df_train, ['tags'])

print(info)

{'tags': {'missing_count': 11, 'percent': 0.12632062471290767}, 'overall': {'removed_rows': 11, 'removed_percent': 0.12632062471290767}}


In [12]:
df_test['tags'] = df_test['tags'].fillna('{}')
df_test['text'] = df_test['text'].fillna('Неизвестно')

**Комментарий:** Принял решение удалить строки, где есть пропуски в столбцах 'tags' и 'text', так как их очень мало, а заполнить их невозможно. В df_test заполнил пропуски заглушками, так как принимается предсказание, которое содержит все строки из тестового набора данных

### Обработка пропусков в столбце 'assessment'

In [13]:
def calculate_missing_percentage(df, column_name):
    total_rows = df.shape[0]
    missing_count = df[column_name].isnull().sum()
    missing_percentage = (missing_count / total_rows) * 100
    print(f'Процент пропусков в столбце "assessment": {missing_percentage:.2f}%')

In [14]:
missing_percentage = calculate_missing_percentage(df_train, 'assessment')

Процент пропусков в столбце "assessment": 2.64%


In [15]:
missing_percentage = calculate_missing_percentage(df_test, 'assessment')

Процент пропусков в столбце "assessment": 2.74%


**Комментарий:** процент пропусков достаточно большой для удаления, поэтому заменим их медианным значением

In [16]:
def fill_missing_with_median(df, column_name):
    median_value = df[column_name].median()
    df[column_name] = df[column_name].fillna(median_value)
    return df

# Заполняем пропуски в 'assessment' для тренировочной и тестовой выборок
df_train = fill_missing_with_median(df_train, 'assessment')
df_test = fill_missing_with_median(df_test, 'assessment')

# Проверяем результаты
print('Пропуски после заполнения для тренировочной выборки:', df_train['assessment'].isnull().sum())
print('Пропуски после заполнения для тестовой выборки:', df_test['assessment'].isnull().sum())

Пропуски после заполнения для тренировочной выборки: 0
Пропуски после заполнения для тестовой выборки: 0


### Нормализация значений 'assessment'

In [17]:
def normalize(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))

    train['assessment'] = scaler.fit_transform(train[['assessment']])
    test['assessment'] = scaler.transform(test[['assessment']])
    return train, test

In [18]:
df_train, df_test = normalize(df_train, df_test)

In [19]:
df_train['assessment'].head()

0    1.000000
1    0.666667
2    1.000000
3    1.000000
4    0.833333
Name: assessment, dtype: float64

In [20]:
df_test['assessment'].head()

0    0.833333
1    0.500000
2    0.833333
3    0.333333
4    0.833333
Name: assessment, dtype: float64

## Обработка текстовых столбцов

**Перед выполнением очистки текста отделим таргет от тренировочной выборки**

In [21]:
target_columns = [f'trend_id_res{i}' for i in range(50)] 


y_train = df_train[target_columns]
X_train = df_train.drop(target_columns, axis=1)

### Преобразование столбца 'tags'

In [22]:
def process_tags(train_df, test_df, column_name='tags'):
 
    mlb = MultiLabelBinarizer()
    
    
    train_tags = [set(tags.strip('{}').split(',')) for tags in train_df[column_name]]
    mlb.fit(train_tags)
    
    
    train_tags_encoded = mlb.transform(train_tags)
    

    test_tags = [set(tags.strip('{}').split(',')) for tags in test_df[column_name]]
    test_tags_encoded = mlb.transform(test_tags)
    
    # Создаем DataFrame с преобразованными столбцами
    train_df_encoded = pd.DataFrame(train_tags_encoded, columns=mlb.classes_)
    test_df_encoded = pd.DataFrame(test_tags_encoded, columns=mlb.classes_)
    

    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    

    train_df = pd.concat([train_df, train_df_encoded], axis=1).drop(columns=[column_name])
    test_df = pd.concat([test_df, test_df_encoded], axis=1).drop(columns=[column_name])
    
    
    return train_df, test_df

In [23]:
X_train, df_test = process_tags(X_train, df_test, column_name='tags')

D:\ANACONDA)))\envs\ds_practicum_env\lib\site-packages\sklearn\preprocessing\_label.py:860: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [24]:
X_train.head()

,index,assessment,text,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT
0,5652,1.000000,"Маленький выбор товаров, хотелось бы ассортиме...",1,0,1,0,0,0,1,0
1,18092,0.666667,Быстро,1,0,1,0,1,1,0,0
2,13845,1.000000,Доставка постоянно задерживается,1,0,1,0,1,0,1,1
3,25060,1.000000,Наценка и ассортимент расстраивают,1,0,0,0,1,0,1,0
4,15237,0.833333,Доставка просто 👍,1,1,0,0,0,1,1,0


In [25]:
df_test.head()

,index,assessment,text,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT
0,5905,0.833333,"Крутая компания, удобное приложение",0,0,0,0,0,0,1,0
1,3135,0.500000,"Последнее время думаю плохо, сроки доставки да...",0,0,1,0,0,0,0,0
2,9285,0.833333,Супер!!!,1,0,0,0,0,0,1,0
3,4655,0.333333,Цены намного выше магазинных но радуют акции,1,0,1,0,1,0,0,0
4,16778,0.833333,Отлично,1,1,1,1,1,1,1,1


## Предобработка 'text'

In [26]:
def cp(train, y, test):
    train_copy = train.copy()
    y_copy = y.copy()
    test_copy = test.copy()
    return train_copy, y_copy, test_copy

In [27]:
nltk.download('stopwords')
nltk.download('punkt')
russian_stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yarma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yarma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
def preprocess_text(text):
    # Очищаем текст от специальных символов и цифр
    text = re.sub(r'[^а-яА-ЯёЁ]', ' ', text.lower())

    # Токенизация
    tokens = word_tokenize(text, language="russian")

    # Удаление стоп слов
    tokens = [token for token in tokens if token not in russian_stop_words]

    # Лемматизация каждого токена
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]

    return ' '.join(lemmas)

def preprocess_data(data, cache_file):
    if os.path.exists(cache_file):
        # Загружаем данные из файла кэша
        with open(cache_file, 'rb') as f:
            lemmatized_data = pickle.load(f)
    else:
        # Применяем предобработку и лемматизацию к данным
        lemmatized_data = [preprocess_text(text) for text in data]
        # Сохраняем обработанные данные в файл кэша
        with open(cache_file, 'wb') as f:
            pickle.dump(lemmatized_data, f)
    
    return lemmatized_data

In [29]:
X_train['text'] = preprocess_data(X_train['text'], 'train_data_lemmatized.pkl')
X_train['text'].head()

0    маленький выбор товар хотеться ассортимент вроде
1                                              быстро
2                    доставка постоянно задерживаться
3                    наценка ассортимент расстраивать
4                                     доставка просто
Name: text, dtype: object

In [30]:
df_test['text'] = preprocess_data(df_test['text'], 'test_data_lemmatized.pkl')
df_test['text'].head()

0                   крутой компания удобный приложение
1    последний время думать плохо срок доставка дав...
2                                                супер
3          цена намного выше магазинный радовать акция
4                                              отлично
Name: text, dtype: object

### Ветроризация с помощью tf-idf

In [31]:
def vectorize_data(train_df, test_df, vectorizer_file='tfidf_vectorizer.pkl'):

    try:
        with open(vectorizer_file, 'rb') as f:
            vectorizer = pickle.load(f)
    except FileNotFoundError:
        vectorizer = TfidfVectorizer(max_features=300, min_df=50, max_df=0.4)
        vectorizer.fit(train_df['text'])
        with open(vectorizer_file, 'wb') as f:
            pickle.dump(vectorizer, f)
    
    # Векторизация текста
    train_tfidf = vectorizer.transform(train_df['text'])
    test_tfidf = vectorizer.transform(test_df['text'])
    
    # Удаление столбца 'text'
    train_df.drop('text', axis=1, inplace=True)
    test_df.drop('text', axis=1, inplace=True)
    
    # Создание нового DataFrame из TF-IDF массива
    train_tfidf_df = pd.DataFrame(train_tfidf.toarray(), 
                                  columns=[f'tfidf_{i}' for i in range(train_tfidf.shape[1])], 
                                  index=train_df.index
                                 )
    test_tfidf_df = pd.DataFrame(test_tfidf.toarray(), 
                                 columns=[f'tfidf_{i}' for i in range(test_tfidf.shape[1])], 
                                 index=test_df.index
                                )
    
    # Объединение TF-IDF DataFrame с исходными DataFrame
    train_df = pd.concat([train_df, train_tfidf_df], axis=1)
    test_df = pd.concat([test_df, test_tfidf_df], axis=1)
    
    return train_df, test_df

In [32]:
X_train_tf, y_train_tf, df_test_tf = cp(X_train, y_train, df_test)
X_train_tf, df_test_tf = vectorize_data(X_train_tf, df_test_tf)

In [33]:
X_train_tf.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,tfidf_195,tfidf_196,tfidf_197,tfidf_198,tfidf_199,tfidf_200,tfidf_201,tfidf_202,tfidf_203,tfidf_204
0,5652,1.000000,1,0,1,0,0,0,1,0,...,0.424546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18092,0.666667,1,0,1,0,1,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13845,1.000000,1,0,1,0,1,0,1,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25060,1.000000,1,0,0,0,1,0,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15237,0.833333,1,1,0,0,0,1,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_test_tf.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,tfidf_195,tfidf_196,tfidf_197,tfidf_198,tfidf_199,tfidf_200,tfidf_201,tfidf_202,tfidf_203,tfidf_204
0,5905,0.833333,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3135,0.500000,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.252338,0.0,0.0,0.0,0.0,0.0,0.0
2,9285,0.833333,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,4655,0.333333,1,0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.288681,0.0,0.0,0.0,0.0,0.0,0.0
4,16778,0.833333,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
X_train_tf.isna().sum().sum()

0

In [36]:
df_test_tf.isna().sum().sum()

0

**Векторизация успешно выполнена**

### Приведение текста к нужной форме (word2vec)

In [37]:
# Маппинг из Natasha в Universal PoS Tags (UPoS)
natasha2upos = {
    'ADJ': 'ADJ',
    'ADV': 'ADV',
    'ADVPRO': 'ADV',
    'ANUM': 'ADJ',
    'APRO': 'DET',
    'COM': 'ADJ',
    'CONJ': 'SCONJ',
    'INTJ': 'INTJ',
    'NONLEX': 'X',
    'NUM': 'NUM',
    'PART': 'PART',
    'PR': 'ADP',
    'S': 'NOUN',
    'SPRO': 'PRON',
    'UNKN': 'X',
    'V': 'VERB'
}

def tag_natasha(text):

    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    morph_vocab = MorphVocab()
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    return [(token.text, natasha2upos.get(token.pos, 'X')) for token in doc.tokens]

def process_text_natasha(text):
    tagged = tag_natasha(text)
    return ' '.join([f"{lemma}_{pos}" for lemma, pos in tagged])

def process_text_column_parallel(dataframe, text_column):
    texts = dataframe[text_column].tolist()
    
    # Используем joblib для параллельной обработки текста
    tagged_texts = Parallel(n_jobs=-1)(delayed(process_text_natasha)(text) for text in texts)
    
    dataframe[text_column] = tagged_texts
    return dataframe

def transform_and_return_datasets(train_data, 
                                  test_data, 
                                  text_column, 
                                  train_filename='train_data_2vec.pkl', 
                                  test_filename='test_data_2vec.pkl'
                                 ):
    # Проверяем, существуют ли файлы с обработанными данными
    if os.path.isfile(train_filename) and os.path.isfile(test_filename):
        print(f"Загружаем обработанные данные из {train_filename} и {test_filename}")
        train_data_processed = pd.read_pickle(train_filename)
        test_data_processed = pd.read_pickle(test_filename)
    else:
        print("Обработанные данные не найдены, начинаем преобразование...")
        train_data_processed = process_text_column_parallel(train_data, text_column)
        test_data_processed = process_text_column_parallel(test_data, text_column)
        
        # Сохраняем обработанные данные
        print(f"Сохранение обработанных данных в {train_filename} и {test_filename}")
        train_data_processed.to_pickle(train_filename)
        test_data_processed.to_pickle(test_filename)
        
    return train_data_processed, test_data_processed

In [38]:
X_train_2vec, y_train_2vec, df_test_2vec = cp(X_train, y_train, df_test)

X_train_2vec, df_test_2vec = transform_and_return_datasets(X_train_2vec, df_test_2vec, 'text')

Загружаем обработанные данные из train_data_2vec.pkl и test_data_2vec.pkl


In [39]:
X_train_2vec.head()

,index,assessment,text,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT
0,5652,1.000000,маленький_ADJ выбор_X товар_X хотеться_ADJ асс...,1,0,1,0,0,0,1,0
1,18092,0.666667,быстро_ADV,1,0,1,0,1,1,0,0
2,13845,1.000000,доставка_X постоянно_ADV задерживаться_X,1,0,1,0,1,0,1,1
3,25060,1.000000,наценка_X ассортимент_X расстраивать_X,1,0,0,0,1,0,1,0
4,15237,0.833333,доставка_X просто_PART,1,1,0,0,0,1,1,0


In [40]:
df_test_2vec.head()

,index,assessment,text,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT
0,5905,0.833333,крутой_ADJ компания_X удобный_ADJ приложение_X,0,0,0,0,0,0,1,0
1,3135,0.500000,последний_ADJ время_X думать_X плохо_ADV срок_...,0,0,1,0,0,0,0,0
2,9285,0.833333,супер_ADV,1,0,0,0,0,0,1,0
3,4655,0.333333,цена_X намного_ADV выше_ADJ магазинный_X радов...,1,0,1,0,1,0,0,0
4,16778,0.833333,отлично_ADV,1,1,1,1,1,1,1,1


### Ветроризация с помощью word2vec

In [41]:
def extend_and_cache_word2vec(train_data, test_data, text_column='text'):
    # Загрузка модели Word2Vec
    model = KeyedVectors.load_word2vec_format('model.bin', binary=True)
       
    # Функция векторизации текста
    def vectorize_text(text, model):
        vectors = np.array([model[word] for word in text.split() if word in model])
        if vectors.any():
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(model.vector_size)
    
    # Создание новых столбцов в DataFrame с названиями 'vector_i'.
    def create_vector_columns(data, model, text_column):
        vector_data = data[text_column].apply(lambda x: vectorize_text(x, model))
        column_names = ['vector_{}'.format(i) for i in range(model.vector_size)]
        new_columns = pd.DataFrame(vector_data.tolist(), index=data.index)
        
        # Важно проверить порядок индексов перед слиянием
        new_columns = new_columns.reindex(data.index)
        data = pd.concat([data.drop(columns=[text_column], axis=1), new_columns], axis=1)
        return data

    train_data = create_vector_columns(train_data, model, text_column)
    test_data = create_vector_columns(test_data, model, text_column)
    
    return train_data, test_data

In [42]:
X_train_2vec, df_test_2vec = extend_and_cache_word2vec(X_train_2vec, df_test_2vec)

In [43]:
X_train_2vec.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,290,291,292,293,294,295,296,297,298,299
0,5652,1.000000,1,0,1,0,0,0,1,0,...,1.056331,2.531532,-0.417632,1.646268,-0.166784,0.914517,-0.720965,-0.950779,-0.736143,-1.661631
1,18092,0.666667,1,0,1,0,1,1,0,0,...,-0.088713,2.783958,-2.259934,1.845464,0.996524,-0.381817,1.599973,-0.321397,0.793367,1.696323
2,13845,1.000000,1,0,1,0,1,0,1,1,...,0.160047,-0.828429,0.964021,-0.907930,0.435427,1.394353,-0.342598,-3.111083,0.188886,0.180546
3,25060,1.000000,1,0,0,0,1,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,15237,0.833333,1,1,0,0,0,1,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
df_test_2vec.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,290,291,292,293,294,295,296,297,298,299
0,5905,0.833333,0,0,0,0,0,0,1,0,...,0.428537,1.045540,-0.930432,-0.009509,-0.166478,-0.551109,2.498223,-1.001278,1.600639,-1.266569
1,3135,0.500000,0,0,1,0,0,0,0,0,...,-0.628026,-0.043070,-0.239992,0.914699,-0.204781,-0.056064,0.394185,0.884897,-0.393142,0.249073
2,9285,0.833333,1,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4655,0.333333,1,0,1,0,1,0,0,0,...,-0.011839,-0.426872,0.764960,-0.381760,-0.334203,-0.945060,0.327364,1.797543,-0.314217,0.414035
4,16778,0.833333,1,1,1,1,1,1,1,1,...,1.133466,-0.173963,-0.761486,1.571459,0.299536,-0.695819,2.084106,0.659884,2.094796,0.299313


### Векторизация с помощью navec

In [45]:
def word2vec_Navec(train_data, test_data, text_column='text'):
    # Загрузка модели Navec
    model = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

    # Получение размерности векторов модели
    vector_size = len(model['любой'])

    # Функция векторизации текста
    def vectorize_text(text, model):
        vectors = []
        for word in text.split():
            if word in model:
                vectors.append(model[word])
        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(vector_size)

    # Создание новых столбцов в DataFrame с названиями 'vector_i'.
    def create_vector_columns(data, model, text_column):
        vector_data = data[text_column].apply(lambda x: vectorize_text(x, model))
        column_names = ['vector_{}'.format(i) for i in range(vector_size)]
        new_columns = pd.DataFrame(vector_data.tolist(), index=data.index, columns=column_names)
        
        # Важно проверить порядок индексов перед слиянием
        new_columns = new_columns.reindex(data.index)
        data = pd.concat([data.drop(columns=[text_column], axis=1), new_columns], axis=1)
        return data

    train_data = create_vector_columns(train_data, model, text_column)
    test_data = create_vector_columns(test_data, model, text_column)

    return train_data, test_data

In [46]:
X_train_nv, y_train_nv, df_test_nv = cp(X_train, y_train, df_test)

In [47]:
X_train_nv, df_test_nv = word2vec_Navec(X_train_nv, df_test_nv)

In [48]:
X_train_nv.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299
0,5652,1.000000,1,0,1,0,0,0,1,0,...,0.121423,-0.100267,0.010518,-0.099883,0.038856,-0.212094,-0.034176,-0.090012,-0.144501,0.175223
1,18092,0.666667,1,0,1,0,1,1,0,0,...,0.143450,-0.201620,0.049041,-0.508832,0.074968,-0.058908,-0.061386,-0.086529,-0.115004,-0.329706
2,13845,1.000000,1,0,1,0,1,0,1,1,...,-0.023868,-0.181181,-0.103018,-0.094392,0.129392,-0.192992,-0.126518,-0.223671,-0.021682,0.045095
3,25060,1.000000,1,0,0,0,1,0,1,0,...,0.151451,-0.380316,0.120812,0.038782,-0.303088,-0.195746,-0.109607,0.082523,0.033192,-0.288488
4,15237,0.833333,1,1,0,0,0,1,1,0,...,-0.011109,-0.276664,-0.122936,0.327358,0.111657,-0.272108,0.290356,-0.259215,0.046564,0.032777


In [49]:
df_test_nv.head()

,index,assessment,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT,...,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299
0,5905,0.833333,0,0,0,0,0,0,1,0,...,-0.006225,-0.048266,-0.204637,0.218369,0.450372,-0.159218,-0.102870,0.000917,0.147460,-0.105937
1,3135,0.500000,0,0,1,0,0,0,0,0,...,0.140696,-0.176247,-0.051177,0.083234,0.055488,-0.166484,0.193623,-0.114053,0.045304,0.362320
2,9285,0.833333,1,0,0,0,0,0,1,0,...,0.510134,0.134358,0.214507,0.416488,-0.073089,-0.378156,0.635190,-0.277134,0.582242,-0.133515
3,4655,0.333333,1,0,1,0,1,0,0,0,...,0.136119,-0.216704,0.244742,0.238277,-0.065249,-0.294463,0.064669,0.072323,-0.023594,0.272356
4,16778,0.833333,1,1,1,1,1,1,1,1,...,-0.105543,-0.513767,-0.148750,0.551946,0.232475,-0.313780,0.580373,-0.091456,0.038903,-0.233373


## Вывод по предобработке данных: 
- Явных дубликатов не найдено
- Данные очищенны от пропусков в столбцах 'tags' и 'text' с минимальными потерями
- Тип данных в столбце 'assessment' переведен из float в Int
- Пропуски в столбце 'assessment' заполеннны медианным значением, так как они составляют до 3% от данных и не смогут исказить их
- От df_train отделили target, чтобы удобнее было проводить лемматизаацию и векторизацию
- Данные в столбце 'tags' закодированы с помощью MultiLabelBinarizer
- Данные в столбце 'text' прошли очиску, лемматизацию и векторизацию с использованием TfidfVectorizer, word2vec и navec. Будем смотреть, какой метод векторизации выдает лучшую метрику
- Данные в столбце 'assessment' нормализированны

# Обучение моделей

**Метрика**

In [50]:
def subset_accuracy(y_true, y_pred):
    # Сравнить каждый экземпляр в y_true и y_pred на полное совпадение
    correct_matches = np.all(y_true == y_pred, axis=1)
    # Подсчитать количество полных совпадений
    accuracy = np.mean(correct_matches)
    return accuracy

custom_scorer = make_scorer(subset_accuracy)

**Разделение данных на выборки делать не будем, так как у нас и так мало данных в тренировочной выборке. Будем пользоваться кросс-валидацией**

### Подбор модели на кросс - валидации

In [51]:
def optimize(objective, model_name):
    # Создание имени файла кэша на основе хеша данных
    data_hash = hashlib.md5(pickle.dumps(X_train_tf) + pickle.dumps(y_train_tf)).hexdigest()
    cache_filename = f"{model_name}_{data_hash}.pkl"
    
    # Проверяем наличие кэша
    if os.path.isfile(cache_filename):
        print(f"Loading cached study for {model_name}")
        with open(cache_filename, "rb") as f:
            study = pickle.load(f)
    else:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, n_jobs=-1)
        
        # Сохранение исследования в кэш
        with open(cache_filename, "wb") as f:
            pickle.dump(study, f)
      
    print('Number of finished trials:', len(study.trials))
    print('Best trial:')
    trial = study.best_trial
    
    print('  Value: {}'.format(trial.value))
    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

### XGBClassifier

In [52]:
def objective_xgb(trial):
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }
    model = XGBClassifier(**params, random_state=random_state)
    
    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)
    accuracy = cross_val_score(model, X_train_tf, y_train_tf, n_jobs=-1, cv=kf, scoring=custom_scorer)
    return np.mean(accuracy)

optimize(objective_xgb, 'XGBClassifier')

Loading cached study for XGBClassifier
Number of finished trials: 10
Best trial:
  Value: 0.6528688053351731
  Params: 
    n_estimators: 794
    max_depth: 3
    learning_rate: 0.08471357596176088
    subsample: 0.7463276476827438
    colsample_bytree: 0.7282951924903549


### LGBMClassifier

In [53]:
def objective_lgbmc(trial):
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }
    base_model = LGBMClassifier(**params, verbose=-1, random_state=random_state)
    model = ClassifierChain(base_model, random_state=random_state)

    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)
    accuracy = cross_val_score(model, X_train_nv, y_train_nv, n_jobs=-1, cv=kf, scoring=custom_scorer)
    return np.mean(accuracy)

optimize(objective_lgbmc, 'LGBMClassifier')

Loading cached study for LGBMClassifier
Number of finished trials: 50
Best trial:
  Value: 0.6783948487984363
  Params: 
    n_estimators: 656
    max_depth: 7
    learning_rate: 0.028575991853669092
    subsample: 0.7088524061710737
    colsample_bytree: 0.6670130277265326


### RandomForestClassifier

In [54]:
def objective_chain_rf(trial):
    base_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }
    base_model = RandomForestClassifier(**base_params, random_state=random_state)
    model = ClassifierChain(base_model, random_state=random_state)

    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)
    accuracy = cross_val_score(model, X_train_tf, y_train_tf, cv=kf, scoring=custom_scorer)
    return np.mean(accuracy)

optimize(objective_chain_rf, 'RandomForestClassifier')

Loading cached study for RandomForestClassifier
Number of finished trials: 10
Best trial:
  Value: 0.571001494768311
  Params: 
    n_estimators: 823
    max_depth: 46
    min_samples_split: 7
    min_samples_leaf: 2


### Нейросеть 

In [55]:
X_train_tf_tensor, y_train_tf_tensor, df_test_tensor = cp(X_train_nv, y_train_nv, df_test_nv)

In [56]:
X_train_tf_tensor = tf.convert_to_tensor(X_train_tf_tensor.values, dtype=tf.float32)
y_train_tf_tensor = tf.convert_to_tensor(y_train_tf_tensor.values, dtype=tf.float32)
df_test_tensor = tf.convert_to_tensor(df_test_tensor.values, dtype=tf.float32)

In [57]:
def custom_accuracy_tensor(y_true, y_pred):
    # Приведение предсказаний к бинарным значениям
    y_pred_bin = tf.round(tf.sigmoid(y_pred))
    # Полное сравнение y_true и y_pred_bin для каждого образца
    correct_predictions = tf.cast(tf.reduce_all(tf.equal(y_true, y_pred_bin), axis=1), tf.float32)
    # Возвращаем долю полных совпадений от общего числа образцов
    return tf.reduce_mean(correct_predictions)

In [ ]:
def create_model(trial, input_dim, output_dim):
    # Строим модель
    model = Sequential()
    model.add(Dense(trial.suggest_int("units_1", 64, 256), input_dim=input_dim, activation='relu'))
    model.add(Dense(trial.suggest_int("units_2", 32, 128), activation='relu'))
    model.add(Dense(output_dim))

    # Компилируем модель с функцией потерь и оптимизатором
    model.compile(optimizer=Adam(learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)),
                  loss=BinaryCrossentropy(from_logits=True),
                  metrics=[custom_accuracy_tensor])
    
    return model

def objective(trial):
    # Автоматический подбор input_dim и output_dim
    input_dim = X_train_tf_tensor.shape[1]
    output_dim = y_train_tf_tensor.shape[1]

    model = create_model(trial, input_dim, output_dim)

    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)

    scores = []
    for train_index, test_index in kf.split(X_train_tf_tensor):
        X_train_fold = tf.gather(X_train_tf_tensor, train_index)
        X_test_fold = tf.gather(X_train_tf_tensor, test_index)
        y_train_fold = tf.gather(y_train_tf_tensor, train_index)
        y_test_fold = tf.gather(y_train_tf_tensor, test_index)

        # Обучаем модель на фолде
        model.fit(X_train_fold, y_train_fold, epochs=50, batch_size=trial.suggest_int("batch_size", 2, 32), verbose=0)

        # Оцениваем модель на тестовом фолде
        score = model.evaluate(X_test_fold, y_test_fold, verbose=0)
        scores.append(score[1])  # Используем custom_accuracy_tensor в качестве метрики

    return np.mean(scores)

def optimize(objective, X_data, y_data, model_name, n_trials=15, n_jobs=-1):
    # Создание имени файла кэша на основе хеша данных
    data_bytes = pickle.dumps(X_data) + pickle.dumps(y_data)
    data_hash = hashlib.md5(data_bytes).hexdigest()
    cache_filename = f"{model_name}_{data_hash}.pkl"
    
    # Проверяем наличие кэша
    if os.path.isfile(cache_filename):
        print(f"Loading cached study for {model_name}")
        with open(cache_filename, "rb") as f:
            study = pickle.load(f)
    else:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        
        # Сохранение исследования в кэш
        with open(cache_filename, "wb") as f:
            pickle.dump(study, f)
            
    print("Лучшие параметры:", study.best_params)
    print("Лучшая метрика:", study.best_value)

# Запуск оптимизации и сохранение результатов в файл с уникальным именем.
optimize(objective, X_train_nv, y_train_nv, 'my_neural_network')

## Вывод по обучению моделей

Самые лучшие результаты у моделей были на tf-idf датасете:

1) XGBClassifier: 0.652
2) LGBMClassifier: 0.678
3) RandomForestClassifier: 0.571
   
Нейросеть показывала на всех рассмотренных мной способах метрику около 0.47. Возможно ей не хватает данных для обучения и\или она неправильно сделана

На способах word2vec и navec 3 основные модели показвали результаты в районе 0.55 - 0.57, скорее всего эти способы не подходят для классических ML моделей

Лучшие результаты выдал LGBMClassifier, попробуем получить более высокую метрику путем расширения сетки параметров модели и подбора параметров самого TfidfVectorizer

## Контрольная оптимизация LGBMClassifier 

In [ ]:
def tf_and_lgb_opt(trial, X_train_tf_ls):

    # Векторизация данных
    vectorizer_params = {
        'max_features': trial.suggest_int('max_features', 100, 300),
        'min_df': trial.suggest_float('min_df', 0.0026, 0.0038),
        'max_df': trial.suggest_float('max_df', 0.5, 0.55, log=True),
        'use_idf': trial.suggest_categorical('use_idf', [True, False]),
        'smooth_idf': trial.suggest_categorical('smooth_idf', [True, False]),
        'sublinear_tf': trial.suggest_categorical('sublinear_tf', [True, False])
    }

    vectorizer = TfidfVectorizer(**vectorizer_params)

    train_tfidf = vectorizer.fit_transform(X_train_tf_ls['text'])

    # Удаление столбца 'text'
    X_train_tf_ls.drop('text', axis=1, inplace=True)

    
    # Создание нового DataFrame из TF-IDF массива
    train_tfidf_df = pd.DataFrame(train_tfidf.toarray(), 
                                  columns=[f'tfidf_{i}' for i in range(train_tfidf.shape[1])], 
                                  index=X_train_tf_ls.index
                                 )
    X_train_tf_ls = pd.concat([X_train_tf_ls, train_tfidf_df], axis=1)
    
    return X_train_tf_ls, vectorizer

def objective(trial):
    X_train_tf_ls = X_train.copy()
    y_train_tf_ls = y_train.copy()
    
    X_train_tf_ls, vectorizer = vectorize_data(trial, X_train_tf_ls)

    model_params = {
        'n_estimators': trial.suggest_int('n_estimators', 650, 1200),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
        'num_leaves': trial.suggest_int('num_leaves', 150, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.2),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5)
    }

    base_model = LGBMClassifier(**model_params, verbose=-1, random_state=random_state)
    model = ClassifierChain(base_model, random_state=random_state)
    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)
    accuracy = cross_val_score(model, X_train_tf_ls, y_train_tf_ls, cv=kf, scoring=custom_scorer, n_jobs=10)

    current_value = np.mean(accuracy)
    return current_value

In [ ]:
# Создание и оптимизация study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=10)

# Вывод результатов оптимизации и сохранение лучших модели и векторизатора
print(f'Number of finished trials: {len(study.trials)}')
best_trial = study.best_trial
print(f'Best trial: {best_trial.number}')
print(f'  Value: {best_trial.value}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

**Комментарий:** Получили чуть более высокую метрику (0.679), идем получать предсказания модели на тесстовой выборке

# Получение предсказаний LGBMClassifier на тестовой выборке

In [59]:
model = LGBMClassifier(n_estimators=780, 
                       max_depth=5, 
                       learning_rate=0.04659066132026324, 
                       subsample=0.8786896310674464, 
                       colsample_bytree=0.5511668094961741, 
                       num_leaves=225,
                       min_child_samples=48,
                       reg_alpha=0.03634162889538072,
                       reg_lambda=0.25095274864196004,
                       verbose=-1, 
                       random_state=random_state
                      )
best_model = ClassifierChain(model, random_state=random_state)
best_model.fit(X_train_tf, y_train_tf)
pred=best_model.predict(df_test_tf)

## Загрузка решения

In [60]:
res = pd.DataFrame(np.hstack([df_test["index"].values.reshape(df_test.shape[0], 1), pred]),
                  columns = ["index"]+[f"trend_id_res{i}" for i in range(50)])

In [61]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16999 entries, 0 to 16998
Data columns (total 51 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           16999 non-null  float64
 1   trend_id_res0   16999 non-null  float64
 2   trend_id_res1   16999 non-null  float64
 3   trend_id_res2   16999 non-null  float64
 4   trend_id_res3   16999 non-null  float64
 5   trend_id_res4   16999 non-null  float64
 6   trend_id_res5   16999 non-null  float64
 7   trend_id_res6   16999 non-null  float64
 8   trend_id_res7   16999 non-null  float64
 9   trend_id_res8   16999 non-null  float64
 10  trend_id_res9   16999 non-null  float64
 11  trend_id_res10  16999 non-null  float64
 12  trend_id_res11  16999 non-null  float64
 13  trend_id_res12  16999 non-null  float64
 14  trend_id_res13  16999 non-null  float64
 15  trend_id_res14  16999 non-null  float64
 16  trend_id_res15  16999 non-null  float64
 17  trend_id_res16  16999 non-null 

In [62]:
res[["index"]+[f"trend_id_res{i}" for i in range(50)]].to_csv("submission.csv", index=False)

# Общий вывод

В ходе проведенного анализа по разработке прототипа модели множественной классификации пользовательских комментариев, была достигнута цель классификации обратной связи по доставке на 50 классов. Работа над проектом включала загрузку и первичный анализ данных, предобработку включая очистку от пропусков и дубликатов, а также преобразование типов данных. Для векторизации текстовых комментариев использовались различные подходы, включая TfidfVectorizer, word2vec и navec, чтобы определить оптимальный метод для последующего применения в моделях машинного обучения.
В данной задаче было принято решение не разделять данные на обучающую и тестовую выборки и использовать кросс-валидацию из-за ограниченного объема данных. В результате обучения моделей лучшие показатели точности демонстрирует LGBMClassifier с результатом 0.678 на данных, векторизированных с помощью TfidfVectorizer. Стоит отметить, что попытки применения нейронной сети не привели к удовлетворительным результатам, что может быть обусловлено как нехваткой данных для обучения, так и возможной недостаточной настройкой архитектуры.
Финальный этап оптимизации LGBMClassifier с расширенной сеткой гиперпараметров позволил достичь незначительного увеличения метрики до 0.679, что является лучшим результатом данного исследования. Ключевым фактором успеха стал выбор правильной стратегии обработки данных и применения именно той методики векторизации, которая позволила максимально эффективно использовать доступные данные для обучения модели.
На основе полученного опыта можно предложить для будущих проектов данного направления обратить внимание на возможность увеличения объема и качества данных, а также исследовать более глубокие и тонкие методы настройки как алгоритмов векторизации, так и самих моделей машинного обучения. Это может привести к дальнейшему повышению точности и надежности классификации пользовательских комментариев.